# Pre & Post purchase FAQs

In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import missingno as msno
import autoreload
import csv
import warnings
import os
import sys

from collections import defaultdict, Counter

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1000)

sb.set()
sb.set_style('darkgrid')

plt.style.use('seaborn')
plt.tight_layout()

%matplotlib inline
%reload_ext autoreload
%autoreload 2

warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv(r"D:\Open Classroom\Datasets\Pre-and-Post Purchase Questions\PrePostQuestions.csv", engine='python')
df.head()

,asin,question,item_name,hours_diff,label
0,B017PICGL0,Are boots cold reliable?,"Reebok Work Men's Zigkick RB7005 Work Shoe, Br...",-1.0,Pre
1,B01HI8YEZS,is the main body rubber or plastic?,"3C-Aone Galaxy S5 Case,Mangix Built-in Glass L...",21.0,Pre
2,B07GZ1LF4R,Do you know the thread pitch/count for the tang?,"ColdLand |14.00"" Hand Forged Damascus Steel Bl...",166.0,Post
3,B07BMNPRFN,The item ordered came tarnished and my son nee...,BEICHUANG Beidou 7 Stars Big Dipper Star Penda...,116.0,Post
4,B075QQKFK3,"""I purchased these and now they look weird wit...",Alla Lighting H8 H11 LED Bulbs Xtreme Super Br...,1021.0,Post


In [3]:
df.shape

(232492, 5)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232492 entries, 0 to 232491
Data columns (total 5 columns):
asin          232492 non-null object
question      232492 non-null object
item_name     232369 non-null object
hours_diff    232492 non-null float64
label         232492 non-null object
dtypes: float64(1), object(4)
memory usage: 8.9+ MB


In [5]:
df.isnull().sum()

asin            0
question        0
item_name     123
hours_diff      0
label           0
dtype: int64

In [6]:
df.drop(columns = ['asin', 'item_name', 'hours_diff'], inplace = True)
df.head()

,question,label
0,Are boots cold reliable?,Pre
1,is the main body rubber or plastic?,Pre
2,Do you know the thread pitch/count for the tang?,Post
3,The item ordered came tarnished and my son nee...,Post
4,"""I purchased these and now they look weird wit...",Post


## Feature Engineering

In [17]:
# Train and Test the dataset
le = LabelEncoder()
x = df['question']

y = df['label']
y = le.fit_transform(y)

le = LabelEncoder()
y = le.fit_transform(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)
print('x train', x_train.shape)
print('x test', x_test.shape)
print('y train', y_train.shape)
print('y test', y_test.shape)

x train (185993,)
x test (46499,)
y train (185993,)
y test (46499,)


In [18]:
# Vectorization - convert text to Document-Term Matrix
vect = TfidfVectorizer(min_df = 3, stop_words='english', max_features=3)

In [19]:
# x_train vectorization 
x_train_vect = vect.fit_transform(x_train)
x_train_vect

<185993x3 sparse matrix of type '<class 'numpy.float64'>'
	with 54050 stored elements in Compressed Sparse Row format>

In [20]:
x_test_vect = vect.transform(x_test)
%time x_test_vect

Wall time: 0 ns


<46499x3 sparse matrix of type '<class 'numpy.float64'>'
	with 13544 stored elements in Compressed Sparse Row format>

In [21]:
# Model Building and Evaluation

dtc = DecisionTreeClassifier()
lr = LinearRegression()

In [26]:
# 1. Decision Tree Classification
dtc.fit(x_train_vect, y_train)
pred = dtc.predict(x_train_vect)
pred

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)